<a href="https://colab.research.google.com/github/dgaida/llm_client/blob/master/notebooks/RAGChatbot_groq_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Chatbot using Retrieval Augmented Generation
# Uses Groq Inference Platform to run LLM online

# TODO: benötigt groq API Key in GROQ_API_KEY und HuggingFace Token in HF_TOKEN (für Embedding Modell)
# TODO: erstelle Ordner pdfs und lade ein oder mehrere PDFs in diesen Ordner hoch

# install packages
!apt-get install -y poppler-utils tesseract-ocr libmagic1
!pip install "unstructured[all-docs]" python-magic llama-index==0.12.35 llama-index-vector-stores-chroma llama-index-embeddings-huggingface chromadb pdf2image pytesseract gradio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
libmagic1 is already the newest version (1:5.41-3ubuntu0.1).
libmagic1 set to manually installed.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.11 [186 kB]
Fetched 186 kB in 1s (325 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━

In [4]:
# install package that lets you use Groq or OpenAI
!git clone https://github.com/dgaida/llm_client.git
%cd llm_client
!pip install .
%cd ..

fatal: destination path 'llm_client' already exists and is not an empty directory.
/content/llm_client/llm_client
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
/content/llm_client


In [2]:
# --- Import required libraries ---
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, StorageContext
from llama_index.readers.file.unstructured import UnstructuredReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Settings
from chromadb.config import Settings as ChromaSettings
import gradio as gr
import os
import chromadb
import random
from llm_client import LLMClient, LLMClientAdapter

# --- Step 1: Set up PDF file path ---
# Put your PDF files in a folder named "pdfs" in the current directory
PDF_DIR = "pdfs"

In [7]:
# --- Step 2: Load and parse PDFs with uMiner (via UnstructuredReader) ---
# --- Check if PDF directory exists ---
if not os.path.exists(PDF_DIR):
    raise FileNotFoundError(f"The folder '{PDF_DIR}' does not exist. Please create it and add PDF files.")

reader = UnstructuredReader()
all_documents = []

# Loop through PDF files in the folder and read each one individually
for filename in os.listdir(PDF_DIR):
    print("File:" + filename)
    if filename.lower().endswith(".pdf"):
        file_path = os.path.join(PDF_DIR, filename)
        docs = reader.load_data(file_path)
        all_documents.extend(docs)

print(f"Loaded {len(all_documents)} documents")
for doc in all_documents[:3]:
    print(doc.text[:300])  # Zeige ersten Ausschnitt

File:Informatik_BSc_Modulhandbuch.pdf


Loaded 1 documents
Modulverzeichnis

Studiengang: Informatik (Bachelor)

Module:

Algorithmen und Programmierung I

BWL I - Grundlagen

Einführung in Betriebssysteme und Rechnerarchitekturen

Mathematik I

Theoretische Informatik I

Algorithmen und Programmierung II

BWL II - Rechnungswesen

Mathematik II

Querschnitt


In [8]:
# split documents into smaller chunks for better retrieval
node_parser = SentenceSplitter(chunk_size=256, chunk_overlap=0)  # 512, 50

In [9]:
# this is the embedding model
# models can be found here: https://huggingface.co/spaces/mteb/leaderboard
embed_model = HuggingFaceEmbedding(model_name = "intfloat/e5-small-v2")
#embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [10]:
# client that uses openAI or Grog API, here groq API
client = LLMClient(api_choice = "groq")

In [11]:
# Adapter erzeugen
adapter = LLMClientAdapter(client=client)

In [12]:
# Apply settings globally - LlamaIndex uses this Settings object internally
Settings.llm = adapter  # None
Settings.embed_model = embed_model
Settings.node_parser = node_parser

In [13]:
# --- Step 4: Create vector store and index ---
# Chroma will store vectors in a temporary database
chroma_client = chromadb.EphemeralClient()

# Get the current collections (if any) and clear them
existing_collections = chroma_client.list_collections()
# Loop through the collections and delete them by name
for collection in existing_collections:
    # Extract the collection name and delete it
    collection_name = collection.name  # Assuming the 'Collection' object has a 'name' attribute
    chroma_client.delete_collection(collection_name)

# unfortunately, if you delete a collection, it still seems to be there. Those
# deletions also do not help. If you really want to delete a database you have
# to restart the session
if 'chroma_collection' in locals():
    print("Deleting existing chroma_collection")
    del chroma_collection

if 'vector_store' in locals():
    print("Deleting existing vector_store")
    del vector_store

if 'index' in locals():
    del index

# add random number because deleting a collection not really deletes it -
# this also does not help...
chroma_collection = chroma_client.create_collection(f"dlml{random.randint(0,100)}")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

# here the documents are stored in the database
index = VectorStoreIndex.from_documents(
    all_documents,
    storage_context=storage_context
)

In [14]:
from huggingface_hub.utils import HfHubHTTPError

# --- Step 5: Create query engine ---
query_engine = index.as_query_engine()

In [15]:
# this function does everyting.
# takes the query of the user and embeds it into the vector space,
# does a semantic search in the vector database (compares vectors)
# returns the text of the most similar vector
# passes this text as context to the LLM together with the query
# retrieves the response of the LLM and returns it as a string
# llama_index does this all internally using the Settings object.
def chat_with_pdf(query, history=None):
    # Frage den Vektor-Index ab, der jetzt automatisch das LLM nutzt
    try:
      response = query_engine.query(query)
    except HfHubHTTPError as e:
      print(e)
      response = "error with Huggingface inference provider"

    return str(response)

In [16]:
print(chat_with_pdf("Welche Module gibt es in dem Studiengang?"))

Die Studierenden wenden im Rahmen eines realitätsnahen Projekts selbstständig erworbene Methoden und Techniken an, arbeiten im Unternehmen im Team und setzen sich mit gesellschaftlichen sowie rechtlichen Fragestellungen auseinander.


In [17]:
# --- Step 7: Build Gradio Interface ---
# This creates a simple web UI for asking questions
chat_ui = gr.ChatInterface(
    fn=chat_with_pdf,
    title="PDF RAG Chatbot",
    description="Ask questions about the content of your PDF documents.",
    theme="default",
    examples=["What is this PDF about?", "Summarize the second section."],
)

chat_ui.launch()

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5577e6a3471a34dc0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
